In [1]:
import wrds, datetime
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

conn = wrds.Connection()
# veervohra
#

C:\Users\devya\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [2]:
rfr = conn.raw_sql('''
SELECT
    mcaldt AS date,
    tmytm AS risk_free_rate
FROM
    crsp.tfz_mth_rf
''')

rfr = rfr[:int(len(rfr)/2)] # the table has two RFRs for each date for some reason so I'm just taking the first half

rfr

,date,risk_free_rate
0,1925-12-31,3.637699
1,1926-01-30,3.673502
2,1926-02-27,3.134354
3,1926-03-31,3.121773
4,1926-04-30,2.724158
...,...,...
1172,2023-08-31,5.356142
1173,2023-09-29,5.365935
1174,2023-10-31,5.370237
1175,2023-11-30,5.345954


In [3]:
import yfinance as yf

def deannualize(annual_rate, periods=365):
    return (1 + annual_rate) ** (1/periods) - 1

# 13 WEEK TREASURY BILL (^IRX)
# Treasury Yield 5 Years (^FVX)
# NYSE Fed Funds Rate Index PR (^NYOMFFX) -- Not working with this code

def get_risk_free_rate():
    annualized = yf.download("^FVX")["Adj Close"] # 3-month us treasury bills rates
    daily = annualized.apply(deannualize) # de-annualize
    return pd.DataFrame({"annualized": annualized, "daily": daily})

rates = get_risk_free_rate()
current = rates.iloc[-1]['annualized']
print(rates.tail())

[*********************100%***********************]  1 of 1 completed

            annualized     daily
Date                            
2024-10-21       3.976  0.004406
2024-10-22       4.005  0.004422
2024-10-23       4.057  0.004450
2024-10-24       4.019  0.004430
2024-10-25       4.051  0.004447


In [4]:
factors = conn.raw_sql('''
    SELECT
        date,
        mktrf AS market_factor,  -- Market factor (excess return on the market)
        smb AS size_factor,  -- Size factor (Small Minus Big)
        hml AS value_factor,  -- Value factor (High Minus Low)
        umd AS momentum_factor,  -- Momentum factor
        rf AS risk_free_rate -- One Month Treasury Bill Rate (daily)
    FROM
        ff.factors_monthly
''')

factors

,date,market_factor,size_factor,value_factor,momentum_factor,risk_free_rate
0,1926-07-01,0.0296,-0.0256,-0.0243,NaN,0.0022
1,1926-08-01,0.0264,-0.0117,0.0382,NaN,0.0025
2,1926-09-01,0.0036,-0.0140,0.0013,NaN,0.0023
3,1926-10-01,-0.0324,-0.0009,0.0070,NaN,0.0032
4,1926-11-01,0.0253,-0.0010,-0.0051,NaN,0.0031
...,...,...,...,...,...,...
1173,2024-04-01,-0.0467,-0.0239,-0.0052,-0.0042,0.0047
1174,2024-05-01,0.0434,0.0078,-0.0166,-0.0002,0.0044
1175,2024-06-01,0.0277,-0.0306,-0.0331,0.0090,0.0041
1176,2024-07-01,0.0124,0.0680,0.0573,-0.0242,0.0045


In [5]:
conn.raw_sql( '''
SELECT permno, MIN(comnam) AS company_name
FROM crsp.stocknames
GROUP BY permno
''').head(30)

,permno,company_name
0,83264,GREIF BROTHERS CORP
1,63618,HINDERLITER ENERGY EQUIP CORP
2,10896,CAMILLE ST MORITZ INC
3,69906,SEIBELS BRUCE GROUP INC
4,79030,GREAT CENTRAL MINES LTD
5,44127,INFORMATION DISPLAYS INC
6,11233,BLOC DEVELOPMENT CORP
7,92865,PROSHARES TRUST II
8,86580,NVIDIA CORP
9,91786,PROSHARES TRUST


In [6]:
# NVIDIA as example to figure out the algorithm

conn.raw_sql('''
SELECT *
FROM crsp.ccmxpf_linktable
WHERE lpermno = 86580;
''')

,gvkey,linkprim,liid,linktype,lpermno,lpermco,usedflag,linkdt,linkenddt
0,117768,P,01,LU,86580.0,16382.0,1.0,1999-01-22,None


In [7]:
nvidia = conn.raw_sql('''
WITH comp_data AS (
    SELECT
        f.gvkey,
        f.datadate AS date,
        f.atq AS total_assets,  -- Total Assets
        f.chq AS cash_holdings,  -- Cash and Short-Term Investments
        f.dlttq + f.dlcq AS total_debt,  -- Total Debt (long-term + short-term debt)
        f.ibq AS earnings,  -- Earnings before extraordinary items
        f.xrdq AS rd_expense,  -- R&D expense
        f.dvpq AS dividends_paid,  -- Dividends paid
        f.prccq * f.cshoq AS market_value  -- Market value (price * shares outstanding)
    FROM
        comp.fundq f
    WHERE
        gvkey = '117768'
        AND f.datadate >= '1986-10-29'  -- Ensure data is after the link start date
),
crsp_data AS (
    SELECT
        permno,
        date,
        ret AS stock_return,  -- Stock returns from CRSP
        prc AS stock_price,
        prc * shrout AS market_value  -- Market value
    FROM
        crsp.msf
    WHERE
        permno = 86580  -- Use the correct permno for gvkey 003243
)
SELECT
    c.gvkey,
    cr.permno,
    c.date,
    cr.stock_price,
    c.cash_holdings,
    c.total_assets,
    c.market_value,
    c.earnings,
    c.total_debt,
    c.rd_expense,
    c.dividends_paid
FROM
    comp_data c
JOIN
    crsp_data cr ON c.date = cr.date
ORDER BY
    c.gvkey, c.date;

''')

nvidia

,gvkey,permno,date,stock_price,cash_holdings,total_assets,market_value,earnings,total_debt,rd_expense,dividends_paid
0,117768,86580,1999-04-30,18.25000,NaN,126.656,5.370610e+02,6.261,3.156,8.785,0.0
1,117768,86580,2000-01-31,37.06250,NaN,202.250,1.152644e+03,14.587,2.748,15.421,0.0
2,117768,86580,2000-07-31,60.00000,NaN,507.183,3.927240e+03,22.522,1.776,20.141,0.0
3,117768,86580,2000-10-31,62.14063,NaN,983.346,4.218727e+03,28.071,301.329,22.023,0.0
4,117768,86580,2001-01-31,51.62500,NaN,1016.427,3.534041e+03,31.057,300.966,26.444,0.0
...,...,...,...,...,...,...,...,...,...,...,...
65,117768,86580,2022-01-31,244.86000,1990.0,44187.000,6.136192e+05,3003.000,11831.000,1466.000,0.0
66,117768,86580,2022-10-31,134.97000,2800.0,40488.000,3.331060e+05,680.000,11904.000,1944.000,0.0
67,117768,86580,2023-01-31,195.37000,3389.0,41182.000,4.817824e+05,1414.000,12031.000,1952.000,0.0
68,117768,86580,2023-07-31,467.29001,5783.0,49555.000,1.153739e+06,6189.000,10954.000,2041.000,0.0


In [8]:
nvidia['quarterly_return'] = nvidia['stock_price'].pct_change(fill_method=None).fillna(0)

nvidia

,gvkey,permno,date,stock_price,cash_holdings,total_assets,market_value,earnings,total_debt,rd_expense,dividends_paid,quarterly_return
0,117768,86580,1999-04-30,18.25000,NaN,126.656,5.370610e+02,6.261,3.156,8.785,0.0,0.000000
1,117768,86580,2000-01-31,37.06250,NaN,202.250,1.152644e+03,14.587,2.748,15.421,0.0,1.030822
2,117768,86580,2000-07-31,60.00000,NaN,507.183,3.927240e+03,22.522,1.776,20.141,0.0,0.618887
3,117768,86580,2000-10-31,62.14063,NaN,983.346,4.218727e+03,28.071,301.329,22.023,0.0,0.035677
4,117768,86580,2001-01-31,51.62500,NaN,1016.427,3.534041e+03,31.057,300.966,26.444,0.0,-0.169223
...,...,...,...,...,...,...,...,...,...,...,...,...
65,117768,86580,2022-01-31,244.86000,1990.0,44187.000,6.136192e+05,3003.000,11831.000,1466.000,0.0,-0.592158
66,117768,86580,2022-10-31,134.97000,2800.0,40488.000,3.331060e+05,680.000,11904.000,1944.000,0.0,-0.448787
67,117768,86580,2023-01-31,195.37000,3389.0,41182.000,4.817824e+05,1414.000,12031.000,1952.000,0.0,0.447507
68,117768,86580,2023-07-31,467.29001,5783.0,49555.000,1.153739e+06,6189.000,10954.000,2041.000,0.0,1.391821


In [9]:
# UNEDITED JUST FROM CHATGPT

import pandas as pd

# Assuming you have a DataFrame `df` with columns: 'stock_return', 'cash_share', 'cash_return'

def calculate_cash_hedged_return(row):
    stock_return = row['stock_return']
    cash_share = row['cash_share']
    cash_return = row['cash_return']
    if cash_share < 1:  # Avoid division by zero
        return (1 / (1 - cash_share)) * (stock_return - cash_share * cash_return)
    else:
        return stock_return  # In case of no cash holdings

In [10]:
# UNEDITED JUST FROM CHATGPT

from statsmodels import regression
import statsmodels.api as sm

def run_factor_regression(portfolio_returns, factors):
    X = sm.add_constant(factors)  # Adding a constant term for intercept
    model = regression.linear_model.OLS(portfolio_returns, X).fit()
    return model.summary()

# Example usage
#factors = df[['market_factor', 'size_factor', 'value_factor', 'momentum_factor']]
#portfolio_returns = df['cash_hedged_return']
#result = run_factor_regression(portfolio_returns, factors)
#print(result)

In [11]:
# UNEDITED JUST FROM CHATGPT

import pandas as pd

# Assuming you already have the following DataFrames: rfr, factors, and citi

# 1. Merge Data
# Merge the company data, risk-free rate, and factor data on the 'date' column
df = nvidia.merge(rfr, on='date', how='inner')
df = df.merge(factors, on='date', how='inner') # NONE OF THE FACTORS DATES LINE UP WITH THE CITI/RFR DATES FOR SOME REASON

# 2. Calculate Excess Returns (Company Stock Return Minus Risk-Free Rate)
df['excess_stock_return'] = df['quarterly_return'] - df['risk_free_rate']

# 3. Hedge Factor Exposures
# You will typically regress the excess stock return on the market factors (market, size, value, and momentum)
# For simplicity, assume a linear regression to calculate betas (hedging coefficients)

import statsmodels.api as sm

# Independent variables: the market, size, value, and momentum factors
X = df[['market_factor', 'size_factor', 'value_factor', 'momentum_factor']]

# Dependent variable: excess stock returns
y = df['excess_stock_return']

# Add a constant term to the independent variables (for the intercept)
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Get the factor betas (coefficients for hedging)
betas = model.params

# 4. Cash-Hedged Return Calculation
# The cash-hedged return is the stock return minus the weighted returns from the factors
# This represents the return net of the market risks captured by the factors.

df['cash_hedged_return'] = df['stock_return'] - (
    betas['const'] + betas['market_factor'] * df['market_factor'] +
    betas['size_factor'] * df['size_factor'] +
    betas['value_factor'] * df['value_factor'] +
    betas['momentum_factor'] * df['momentum_factor']
)

# 5. Analyze the Cash-Hedged Returns
# Now you can analyze the performance of the stock after hedging against factor risks.
df[['date', 'cash_hedged_return']].set_index('date').plot(title='Cash-Hedged Stock Returns')

KeyError: 'risk_free_rate'